# Example of pipeline of work with Steganografic Detector

In [1]:
! pip install transformers

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import IMDB
from torchtext.data import Field, LabelField, BucketIterator, TabularDataset

import spacy

import numpy as np

import random
import math
import time

from tqdm import tqdm

from sklearn.metrics import roc_auc_score

import json

SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

! python -m spacy download en

spacy_en = spacy.load('en')

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [303]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

# should be 0 if the sentence is natural and 1 if this is encoded
LABEL = LabelField(dtype = torch.float)

Loading a custom dataset

In [304]:
fields = {'text': ("text",SRC), 'label': ("label",LABEL)}

In [ ]:
gen_data, = TabularDataset.splits(
                            path = '.',
                            root=".",
                            train = './data/attacker_dataVLC_2.json',
                            format = 'json',
                            fields = fields
)

In [306]:
train_data, valid_data = gen_data.split(random_state = random.seed(SEED))

In [307]:
MAX_VOCAB_SIZE = 15_000

SRC.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [308]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [309]:
train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False,
    device = device)


In [ ]:
from model import RNNStegaDetector

In [311]:
model = RNNStegaDetector(batch_size=BATCH_SIZE, 
            output_size=1, 
            hidden_size=100, 
            vocab_size=len(SRC.vocab), 
            n_layers=2,
            embedding_length=80,
            pad_idx=SRC.vocab.stoi[SRC.pad_token], 
            dropout=0.5, 
            bidirectional=True)

In [312]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,587,721 trainable parameters


In [313]:
pretrained_embeddings = SRC.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([15004, 100])


Training procedure for the detector

In [314]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:  
        optimizer.zero_grad()

        # print(batch.text)
        # text, text_lengths = batch.text
        text = batch.text

        predictions = model(text).squeeze(1)

        loss = criterion(predictions, batch.label)

        rounded_preds = torch.round(torch.sigmoid(predictions))
        correct = (rounded_preds == batch.label).float()
        acc = correct.sum() / len(correct)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [315]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    roc_auc = 0
    all_target = []
    all_pred_probs = []

    model.eval()

    with torch.no_grad():

        for batch in iterator: 
            text = batch.text
            predictions = model(text).squeeze(1)

            loss = criterion(predictions, batch.label)

            rounded_preds = torch.round(torch.sigmoid(predictions))
            correct = (rounded_preds == batch.label).float()
            acc = correct.sum() / len(correct)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
            all_target.extend(batch.label.tolist())
            all_pred_probs.extend(torch.sigmoid(predictions).tolist())


    roc_auc = roc_auc_score(all_target, all_pred_probs)

    return epoch_loss / len(iterator), epoch_acc / len(iterator), roc_auc

In [316]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

optimizer = torch.optim.Adam(model.parameters())

In [117]:
import warnings
warnings.filterwarnings('ignore')

In [317]:
#vlc2
N_EPOCHS = 30

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, valid_roc_auc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    # epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {end_time - start_time} s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | ROC AUC:{valid_roc_auc:.3f}')
    
model = model.eval()    
torch.save(model, 'models/attacker_vlc2.pth')

Epoch: 01 | Epoch Time: 6.007275819778442 s
	Train Loss: 0.654 | Train Acc: 68.82%
	 Val. Loss: 0.525 |  Val. Acc: 76.35% | ROC AUC:0.928
Epoch: 02 | Epoch Time: 5.981096982955933 s
	Train Loss: 0.363 | Train Acc: 85.70%
	 Val. Loss: 0.319 |  Val. Acc: 88.70% | ROC AUC:0.964
Epoch: 03 | Epoch Time: 5.819183826446533 s
	Train Loss: 0.267 | Train Acc: 89.74%
	 Val. Loss: 0.297 |  Val. Acc: 89.38% | ROC AUC:0.978
Epoch: 04 | Epoch Time: 6.15967583656311 s
	Train Loss: 0.185 | Train Acc: 93.29%
	 Val. Loss: 0.158 |  Val. Acc: 95.68% | ROC AUC:0.991
Epoch: 05 | Epoch Time: 5.949137210845947 s
	Train Loss: 0.242 | Train Acc: 91.04%
	 Val. Loss: 0.307 |  Val. Acc: 87.50% | ROC AUC:0.982
Epoch: 06 | Epoch Time: 6.439775228500366 s
	Train Loss: 0.185 | Train Acc: 93.43%
	 Val. Loss: 0.164 |  Val. Acc: 94.53% | ROC AUC:0.990
Epoch: 07 | Epoch Time: 6.23667049407959 s
	Train Loss: 0.121 | Train Acc: 96.22%
	 Val. Loss: 0.547 |  Val. Acc: 87.55% | ROC AUC:0.873
Epoch: 08 | Epoch Time: 5.5827565193

In [ ]:
(93.02, 0.987)